## Approximate Nearest Neighbors using LSH

This notebook will show the give and take between building multiple hash tables and employing multiple hash functions when searching for approximate neighbors using LSH. The demonstration is meant to clarify slide 12 from slide deck 3, on Nearest Neighbors.

I have written a basic LSH implementation in Python, with instantiations for the cosine similarity, Hamming distance, and the Jaccard coefficient LSH families. The code is written in OOP style and can be easily extended to other LSH families. Take a look at lsh.py for the details. In this demo, we will be using the cosine similarity version of the LSH data structure, *clsh*.

In [2]:
%load_ext autoreload
%autoreload 2
import time
import numpy as np
from lsh import clsh, jlsh, generateSamples, findNeighborsBrute, recall

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


First, we will generate some random samples, and split the data into train (X) and test (Y) subsets. Samples are generated from 100 gausian blobs, i.e., points will be fairly spread out as far as their cosine similarity is concerned.

In [3]:
# using Jaccard Coefficient
def get_samples(is_binary=False):
    X, Y = generateSamples(nsamples=1000, nfeatures=100, nclusters=64, clusterstd=50, binary=is_binary)
    return X, Y

X, Y = get_samples(is_binary=False)
print(X.shape, Y.shape)

((900, 100), (100, 100))


The basic concept in LSH is that of *hashing* the vectors using a random LSH family of hash functions. As we discussed in class, the LSH families will be more likely to assign the same hash value to similar items. This, however, does not happen all the time. First, let's see what the result of hashing a vector looks like.

In [4]:
L11 = clsh(X, ntables=1, nfunctions=1)
for i in range(10):
    print(L11.hash(X[i,:]))

0
0
0
1
1
1
1
1
1
0


As shown in slide 18, the output of the cosine family of LSH function is binary, depending on the sign of the dot-product $\langle r,x\rangle$ between the random unit vector $r$ and our input vector $x=X[i,:]$.

Note that we created a single table in our LSH data structure and are using a single LSH function to hash vectors. This means that we're simply partitioning vectors into two buckets. Some vectors will go to the bucket with ID 0, and others will go to the bucket with ID 1.

When we instantiated the LSH data structure *L*, all the vectors in X were already assigned to their respective buckets. Let's see how many vectors each bucket has.

In [5]:
print("Bucket ID 0 has %d vectors." % len(L11.tables[0]['0']))
print("Bucket ID 1 has %d vectors." % len(L11.tables[0]['1']))

Bucket ID 0 has 442 vectors.
Bucket ID 1 has 458 vectors.


Now, when it's time to find neighbors for a new vector, say $y=Y[0,:]$, the first vector in our test set, we hash the vector to see which bucket we should look in to find neighbors.

In [6]:
print(L11.hash(Y[0,:], tid=0, fid=0))

0


Note that I passed in the ID of the table I'm searching in and the ID of the function I'm hashing with. For LSH to work, we have to use the same hashing functions that were used to create the table(s). Therefore, $clsh$ stores the randomly generated functions it created for each table.

Now, it looks like I have to compare $y$ against almost half of the vectors in $X$, which is a lot, and leads to low *precision*. Precision is the fraction of retrieved instances (the vectors we compared against) that are relevant (that would also be in the exact result). Since the number of objects we're comparing against is high, precision will be low. In order to increase the precision, I can use several hash functions and concatenate their results. Increasing the precision will also reduc the amount of time spent finding neighbors, as we will have fewer objects to compare against.

Let's say I use 2 hash functions from the Cosine LSH family. Then, the possible resulting hash values would be 00, 01, 10, and 11, spliting the vectors in X into 4 buckets (instead of 2, when we used 1 function). If we use 3 functions, we get 8 buckets. In general, using $f$ functions will split the "search space" into $2^f$ buckets.

Let's try this using 3 functions.

In [7]:
L13 = clsh(X, ntables=1, nfunctions=3)
for k in L13.tables[0].keys():
    print("Bucket ID %s has %d vectors." % (k, len(L13.tables[0][k])))
    
print("\nWe only need to compare y against vectors in bucket %s." % L13.signature(Y[0,:], tid=0))

Bucket ID 010 has 106 vectors.
Bucket ID 011 has 115 vectors.
Bucket ID 001 has 106 vectors.
Bucket ID 000 has 115 vectors.
Bucket ID 111 has 87 vectors.
Bucket ID 110 has 121 vectors.
Bucket ID 100 has 118 vectors.
Bucket ID 101 has 132 vectors.

We only need to compare y against vectors in bucket 101.


**Side note**: Note that in this academic LSH implementation we use a simple way to generate bucket IDs. We concatenate the string representation of the resulting hash value from each hash function. LSH libraries often implement a secondary (exact) hash function for generating numeric IDs for the buckets. A similar scheme is proposed in the LSH reference I nored on Canvas: [SPM'08] Malcolm Slaney and Michael Casey. Locality-Sensitive Hashing for Finding Nearest Neighbors. Lecture Notes. IEEE Signal Processing Magazine, 2008.

It is easy to see we now have much fewer vectors to compare against when we search for $y$'s neighbors. However, some of the true neighbors may have been accidentally placed in other buckets, which lowers *recall*. Recall (also known in Statistics references as *sensitivity*) is the fraction of relevant instances that are retrieved, i.e., the fraction of true neighbors in our top-$k$ divided by $k$. 

Let's compare the mean recall for finding neighbors using 1 hash function vs. 3 hash functions. To do that, we will first have to find the "true neighbors".

In [8]:
k = 100  # number of neighbors to find
nbrsExact = findNeighborsBrute(X, Y, k=k, sim="cos")
print("Number of computed similarities for the brute-force approach: %d." % (X.shape[0] * Y.shape[0]))
nbrsTest11  = L11.findNeighbors(Y, k=k)
nbrsTest13  = L13.findNeighbors(Y, k=k)
print("Recall with 1 hash function: %f. Number of computed similarities: %d." % (recall(nbrsTest11, nbrsExact), L11.nsims))
print("Recall with 3 hash functions: %f. Number of computed similarities: %d." % (recall(nbrsTest13, nbrsExact), L13.nsims))

Number of computed similarities for the brute-force approach: 90000.
Recall with 1 hash function: 0.542100. Number of computed similarities: 45112.
Recall with 3 hash functions: 0.163800. Number of computed similarities: 11467.


We can increase the recall by building several LSH tables instead of one. Then, instead of looking in one bucket for $y$'s neighbors, we will be looking in one bucket in each table. The search method gets the set union of object IDs in all these buckets, and then computes similarities against all of them.

### Excercise 1

Compare the mean recall for finding neighbors using 1 table vs. 3 tables, when each table uses 3 hash functions.

In [9]:
k = 100  # number of neighbors to find
def excercise_1(X, Y, similarity='cos'):
    L33 = clsh(X, ntables=3, nfunctions=3)
    nbrsExact = findNeighborsBrute(X, Y, k=k, sim=similarity)
    print("Number of computed similarities for the brute-force approach: %d." % (X.shape[0] * Y.shape[0]))
    nbrsTest13  = L13.findNeighbors(Y, k=k)
    nbrsTest33  = L33.findNeighbors(Y, k=k)
    print("Recall with 1 tables, 3 hash function: %f. Number of computed similarities: %d." % (recall(nbrsTest13, nbrsExact), L13.nsims))
    print("Recall with 3 tables, 3 hash functions: %f. Number of computed similarities: %d." % (recall(nbrsTest33, nbrsExact), L33.nsims))
    
excercise_1(X, Y, similarity='cos')

Number of computed similarities for the brute-force approach: 90000.
Recall with 1 tables, 3 hash function: 0.163800. Number of computed similarities: 11467.
Recall with 3 tables, 3 hash functions: 0.421700. Number of computed similarities: 29560.


Given high enough # tables and # hashes (hash functions), we can achieve high recall and precision, sometimes at the expense of efficiency.

### Excercise 2

Find the minimum number of tables necessary to obtain `recall` of at least `0.90` using 3 functions. What is the number of computed similarities for that LSH forest?

In [10]:
def excercise_2(X, Y, similarity='cos'):
    if similarity == 'cos':
        name_of_metrics = 'Cosine Similarity'
        
    elif similarity == 'jac':
        name_of_metrics = 'Jaccard coefficient'
    
    print('=========== ' + name_of_metrics + ' ===========')
    
    n_tables = 1

    Ln3 = clsh(X, ntables=n_tables, nfunctions=3)
    if similarity == 'jac':
        Ln3 = jlsh(X, ntables=n_tables, nfunctions=3)

    n_neighbors_exact_n3 = findNeighborsBrute(X, Y, k=k, sim=similarity)

    n_neighbors_test_n3 = Ln3.findNeighbors(Y, k=k)

    recall_rate = recall(n_neighbors_test_n3, n_neighbors_exact_n3)

    while recall_rate < 0.9:
        if similarity == 'jac':
            Ln3 = jlsh(X, ntables=n_tables, nfunctions=3)
        else:
            Ln3 = clsh(X, ntables=n_tables, nfunctions=3)

        n_neighbors_test_n3 = Ln3.findNeighbors(Y, k=k)

        recall_rate = recall(n_neighbors_test_n3, n_neighbors_exact_n3)

        print('Number of tables: ', n_tables)
        print('Recall rate: ', recall_rate)

        n_tables += 1
    print('********** end **********')
    
excercise_2(X, Y, similarity='cos')

=========== Cosine Similarity ===========
('Number of tables: ', 1)
('Recall rate: ', 0.1739000000000001)
('Number of tables: ', 2)
('Recall rate: ', 0.3171999999999999)
('Number of tables: ', 3)
('Recall rate: ', 0.4355000000000001)
('Number of tables: ', 4)
('Recall rate: ', 0.5190000000000001)
('Number of tables: ', 5)
('Recall rate: ', 0.6058000000000002)
('Number of tables: ', 6)
('Recall rate: ', 0.6659)
('Number of tables: ', 7)
('Recall rate: ', 0.7280000000000002)
('Number of tables: ', 8)
('Recall rate: ', 0.7774999999999999)
('Number of tables: ', 9)
('Recall rate: ', 0.8129000000000004)
('Number of tables: ', 10)
('Recall rate: ', 0.8429000000000001)
('Number of tables: ', 11)
('Recall rate: ', 0.8738000000000004)
('Number of tables: ', 12)
('Recall rate: ', 0.8912000000000003)
('Number of tables: ', 13)
('Recall rate: ', 0.9118999999999999)
********** end **********


### Exercise 3

Repeat Exrcise 1 and 2 using `Jaccard Coefficient` instead of `cosine similarity`. Note that you will need to re-generate samples using the `binary=True` parameter and re-compute `nbrsExact` for the new similarity measure.

In [11]:
X , Y = get_samples(is_binary=True)
excercise_1(X, Y, similarity='jac')
print ''
excercise_2(X, Y, similarity='jac')

Number of computed similarities for the brute-force approach: 90000.
Recall with 1 tables, 3 hash function: 0.116700. Number of computed similarities: 11474.
Recall with 3 tables, 3 hash functions: 0.782300. Number of computed similarities: 64664.

=========== Jaccard coefficient ===========
('Number of tables: ', 1)
('Recall rate: ', 0.08459999999999995)
('Number of tables: ', 2)
('Recall rate: ', 0.14489999999999997)
('Number of tables: ', 3)
('Recall rate: ', 0.23339999999999989)
('Number of tables: ', 4)
('Recall rate: ', 0.26629999999999987)
('Number of tables: ', 5)
('Recall rate: ', 0.31279999999999997)
('Number of tables: ', 6)
('Recall rate: ', 0.36439999999999984)
('Number of tables: ', 7)
('Recall rate: ', 0.44299999999999984)
('Number of tables: ', 8)
('Recall rate: ', 0.4664000000000001)
('Number of tables: ', 9)
('Recall rate: ', 0.5545)
('Number of tables: ', 10)
('Recall rate: ', 0.6019)
('Number of tables: ', 11)
('Recall rate: ', 0.6015000000000001)
('Number of tables